In [9]:
import os
import time

In [10]:
#Size of the file
os.path.getsize('C:/Users/mizoh/Desktop/Data Glacier/Data Ingestion/yellow_tripdata.csv')

125457064

In [11]:
#Read data with Dask:
from dask import dataframe as dd
start=time.time()
dask_df=dd.read_csv('C:/Users/mizoh/Desktop/Data Glacier/Data Ingestion/yellow_tripdata.csv')
end=time.time()
print('Read the csv file with dask:',(end-start),'sec')

Read the csv file with dask: 0.022143125534057617 sec


In [12]:
#Read data with Pandas:
import pandas as pd
start=time.time()
pandas_df=pd.read_csv('C:/Users/mizoh/Desktop/Data Glacier/Data Ingestion/yellow_tripdata.csv')
end=time.time()
print('Read te csv file with pandas :',(end-start),'sec')

Read te csv file with pandas : 2.100923776626587 sec


In [13]:
#Read data with modin and ray:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start=time.time()
pandas_df=pd.read_csv('C:/Users/mizoh/Desktop/Data Glacier/Data Ingestion/yellow_tripdata.csv')
end=time.time()
print('Read te csv file with modin and ray :',(end-start),'sec')

2021-08-28 13:06:51,213	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265


Read te csv file with modin and ray : 2.4228675365448 sec


In [14]:
#So we can notice that Dask is better than Pandas,Modin,and Ray with least reading time 0.015

In [15]:
from dask import dataframe as dd
df=dd.read_csv('C:/Users/mizoh/Desktop/Data Glacier/Data Ingestion/yellow_tripdata.csv',delimiter=',')

In [16]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 19 entries, VendorID to total amount
dtypes: object(3), float64(12), int64(4)

In [17]:
len(df.index)

1048575

In [18]:
df.columns

Index(['VendorID', 'tpep pickup datetime', 'tpep dropoff datetime',
       'passenger count', 'trip distance', 'pickup longitude',
       'pickup latitude', 'Rate CodeID', 'store and fwd flag',
       'dropoff longitude', 'dropoff latitude', 'payment type', 'fare amount',
       'extra', 'mta tax', 'tip amount', 'tolls amount',
       'improvement surcharge', 'total amount'],
      dtype='object')

In [19]:
#Remove spaces from the Column Names:
df.columns=df.columns.str.replace(" ",'_')

In [20]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,Rate_CodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,3/6/2015 8:02,3/6/2015 8:09,1,1.2,-73.990211,40.750969,1,N,-73.987892,40.738037,2,7.0,0.0,0.5,0.00,0.0,0.3,7.80
1,1,3/6/2015 8:02,3/6/2015 8:15,1,3.2,-73.935188,40.800720,1,N,-73.952553,40.765373,2,11.5,0.0,0.5,0.00,0.0,0.3,12.30
2,1,3/6/2015 8:02,3/6/2015 8:12,1,1.1,-73.963753,40.767937,1,N,-73.956947,40.780270,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80
3,1,3/6/2015 8:02,3/6/2015 8:09,1,0.8,-73.997177,40.742168,1,N,-74.008064,40.739281,1,6.0,0.0,0.5,1.00,0.0,0.3,7.80
4,1,3/6/2015 8:02,3/6/2015 8:19,1,2.7,-74.006844,40.730267,1,N,-73.976860,40.750671,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55


In [21]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'Rate_CodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [102]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [103]:
%%writefile data.yaml
file_type: csv
dataset_name: file
file_name: yellow_tripdata
table_name: edsurv
inbound_delimiter: ','
outbound_delimiter: '|'
skip_leading_rows: 1
columns:
    - VendorID
    - tpep_pickup_datetime
    - tpep_dropoff_datetime
    - passenger_count
    - trip_distance
    - pickup_longitude
    - pickup_latitude
    - Rate_CodeID
    - store_and_fwd_flag
    - dropoff_longitude
    - dropoff_latitude
    - payment_type
    - fare_amount
    - extra
    - mta_tax
    - tip_amount
    - tolls_amount
    - improvement_surcharge
    - total_amount

Overwriting data.yaml


In [104]:
#Reading Config File:
import utility as util
config_data=util.read_config_file('data.yaml')

In [105]:
#data of config file
config_data['file_type']

'csv'

In [106]:
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'yellow_tripdata',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['VendorID',
  'tpep_pickup_datetime',
  'tpep_dropoff_datetime',
  'passenger_count',
  'trip_distance',
  'pickup_longitude',
  'pickup_latitude',
  'Rate_CodeID',
  'store_and_fwd_flag',
  'dropoff_longitude',
  'dropoff_latitude',
  'payment_type',
  'fare_amount',
  'extra',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'improvement_surcharge',
  'total_amount']}

In [107]:
# Reading process of the file using Dask
from dask import dataframe as dd
df_sample = dd.read_csv('C:/Users/mizoh/Desktop/Data Glacier/Data Ingestion/yellow_tripdata.csv',delimiter=',')
df_sample.head()

,VendorID,tpep pickup datetime,tpep dropoff datetime,passenger count,trip distance,pickup longitude,pickup latitude,Rate CodeID,store and fwd flag,dropoff longitude,dropoff latitude,payment type,fare amount,extra,mta tax,tip amount,tolls amount,improvement surcharge,total amount
0,1,3/6/2015 8:02,3/6/2015 8:09,1,1.2,-73.990211,40.750969,1,N,-73.987892,40.738037,2,7.0,0.0,0.5,0.00,0.0,0.3,7.80
1,1,3/6/2015 8:02,3/6/2015 8:15,1,3.2,-73.935188,40.800720,1,N,-73.952553,40.765373,2,11.5,0.0,0.5,0.00,0.0,0.3,12.30
2,1,3/6/2015 8:02,3/6/2015 8:12,1,1.1,-73.963753,40.767937,1,N,-73.956947,40.780270,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80
3,1,3/6/2015 8:02,3/6/2015 8:09,1,0.8,-73.997177,40.742168,1,N,-74.008064,40.739281,1,6.0,0.0,0.5,1.00,0.0,0.3,7.80
4,1,3/6/2015 8:02,3/6/2015 8:19,1,2.7,-74.006844,40.730267,1,N,-73.976860,40.750671,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55


In [108]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = "C:/Users/mizoh/Desktop/Data Glacier/Data Ingestion/" + config_data['file_name'] + f'.{file_type}'

In [109]:
import pandas as pd
df=pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,VendorID,tpep pickup datetime,tpep dropoff datetime,passenger count,trip distance,pickup longitude,pickup latitude,Rate CodeID,store and fwd flag,dropoff longitude,dropoff latitude,payment type,fare amount,extra,mta tax,tip amount,tolls amount,improvement surcharge,total amount
0,1,3/6/2015 8:02,3/6/2015 8:09,1,1.2,-73.990211,40.750969,1,N,-73.987892,40.738037,2,7.0,0.0,0.5,0.00,0.0,0.3,7.80
1,1,3/6/2015 8:02,3/6/2015 8:15,1,3.2,-73.935188,40.800720,1,N,-73.952553,40.765373,2,11.5,0.0,0.5,0.00,0.0,0.3,12.30
2,1,3/6/2015 8:02,3/6/2015 8:12,1,1.1,-73.963753,40.767937,1,N,-73.956947,40.780270,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80
3,1,3/6/2015 8:02,3/6/2015 8:09,1,0.8,-73.997177,40.742168,1,N,-74.008064,40.739281,1,6.0,0.0,0.5,1.00,0.0,0.3,7.80
4,1,3/6/2015 8:02,3/6/2015 8:19,1,2.7,-74.006844,40.730267,1,N,-73.976860,40.750671,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55


In [110]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['VendorID', 'tpep pickup datetime', 'tpep dropoff datetime',
       'passenger count', 'trip distance', 'pickup longitude',
       'pickup latitude', 'Rate CodeID', 'store and fwd flag',
       'dropoff longitude', 'dropoff latitude', 'payment type', 'fare amount',
       'extra', 'mta tax', 'tip amount', 'tolls amount',
       'improvement surcharge', 'total amount'],
      dtype='object')
columns of YAML are: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'Rate_CodeID', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']


In [112]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('C:/Users/mizoh/Desktop/Data Glacier/Data Ingestion/yellow_tripdata.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('yellow_tripdata.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['C:/Users/mizoh/Desktop/yellow_tripdata.csv.gz\\0.part',
 'C:/Users/mizoh/Desktop/yellow_tripdata.csv.gz\\1.part']

In [115]:
#number of files in gz format folder
import os
entries = os.listdir('C:/Users/mizoh/Desktop/yellow_tripdata.csv.gz/')
for entry in entries:
    print(entry)

0.part
1.part


In [118]:
#size of the gz format folder
os.path.getsize('C:/Users/mizoh/Desktop/yellow_tripdata.csv.gz/')

0